In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Flatten, Dense, BatchNormalization, Conv2D, MaxPooling2D, Dropout
import numpy as np
import pandas as pd
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import os
from PIL import Image

2023-04-14 21:21:57.602903: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-14 21:21:58.464057: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class BirdDataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, use_rows, batch_size, csv_file="birds.csv"):
        self.csv_df = pd.read_csv(csv_file)
        self.use_rows = use_rows
        self.batch_size = batch_size 
        self.dataset = pd.DataFrame(self.csv_df).to_numpy()
    
        self.labels = self.dataset[:,2]
        self.img_filepaths = self.dataset[:,1]
        self.class_ids = self.dataset[:,0]
        
        (x,) = self.img_filepaths.shape
        image = Image.open(self.img_filepaths[0])
        image_array = np.expand_dims(np.asarray(image), axis=0)

    
    def __len__(self):
        # batches_per_epoch is the total number of batches used for one epoch
        batches_per_epoch = int(len(self.use_rows) / self.batch_size)
        return batches_per_epoch
    
    def __getitem__(self, index):
        # index is the index of the batch to be retrieved
        batch_ids = self.use_rows[index * self.batch_size: (index + 1) * self.batch_size]
        
        x = None
        y = None
        
        for curr_id in batch_ids:
            image = Image.open(self.img_filepaths[curr_id]).resize((224,224))
            image_array = np.array(image)

            image_array = np.expand_dims(np.asarray(image),axis=0)
            
            if x is None:
                x = image_array
                y = self.class_ids[curr_id]
            else:
                x = np.vstack((x, image_array))
                y = np.vstack((y, self.class_ids[curr_id]))                
        
        
        y = tf.keras.utils.to_categorical(y, num_classes=511)

        return x, y            
            
    def on_epoch_end(self):
        np.random.shuffle(self.split_ids)
            
            

In [3]:
use_rows = list(range(0,87050))
bird_data_generator = BirdDataGenerator(csv_file="birds.csv", use_rows=use_rows, batch_size=128)
bird_data_generator.__getitem__(0)

(87050, 5)
train/ABBOTTS BABBLER/001.jpg
(1, 224, 224, 3)


(array([[[[180, 172, 149],
          [186, 178, 155],
          [186, 180, 158],
          ...,
          [181, 180, 198],
          [179, 180, 200],
          [178, 179, 199]],
 
         [[182, 174, 153],
          [190, 182, 161],
          [190, 184, 162],
          ...,
          [182, 181, 199],
          [176, 177, 197],
          [178, 179, 199]],
 
         [[186, 178, 159],
          [195, 187, 168],
          [196, 189, 170],
          ...,
          [184, 183, 199],
          [180, 179, 197],
          [183, 182, 200]],
 
         ...,
 
         [[151, 138, 121],
          [172, 159, 142],
          [178, 165, 146],
          ...,
          [189, 190, 211],
          [191, 189, 210],
          [169, 167, 188]],
 
         [[156, 140, 124],
          [155, 139, 123],
          [161, 148, 131],
          ...,
          [186, 188, 211],
          [182, 185, 204],
          [168, 171, 188]],
 
         [[157, 141, 125],
          [156, 140, 124],
          [167, 154, 137],
   

In [4]:
def bird_cnn_model(csv_file):
    model = Sequential([
        InputLayer(input_shape=(224,224,3)),
        
        Conv2D(filters=8, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=2),
        BatchNormalization(),
        Flatten(),
        Dense(128, activation="relu"),
        Dense(128, activation="relu"),
        Dense(511, activation="softmax")
    ])
    
    optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
    model.compile(loss="categorical_crossentropy", optimizer = optimizer, metrics=["accuracy"])
    
    print(model.summary())
    train_rows = list(range(0,81950))
    train_generator = BirdDataGenerator(csv_file=csv_file, use_rows=train_rows, batch_size=64)
    validation_rows = list(range(84500, 87050))
    validation_generator = BirdDataGenerator(csv_file=csv_file, use_rows=validation_rows, batch_size=64)
    h = model.fit(x=train_generator, epochs=1, verbose=1, validation_data = validation_generator)
    print(h)
    
    test_rows = list(range(81950, 84500))
    test_generator = BirdDataGenerator(csv_file=csv_file, use_rows=test_rows, batch_size=64)
    model.evaluate(x=test_generator)
    
    return model
    

In [5]:
cnn_model = bird_cnn_model("birds.csv")

2023-04-14 21:22:01.645906: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-14 21:22:01.710212: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-14 21:22:01.711129: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-14 21:22:01.716255: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-14 21:22:01.717853: I tensorflow/compile

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 8)       224       
                                                                 
 batch_normalization (BatchN  (None, 222, 222, 8)      32        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 8)      0         
 )                                                               
                                                                 
 batch_normalization_1 (Batc  (None, 111, 111, 8)      32        
 hNormalization)                                                 
                                                                 
 flatten (Flatten)           (None, 98568)             0         
                                                        

r/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-14 21:22:02.877068: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-14 21:22:02.877739: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-14 21:22:02.877787: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1722] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.
2023-04-14 21:22:02.878803: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to r

 dense (Dense)               (None, 128)               12616832  
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 511)               65919     
                                                                 
Total params: 12,699,551
Trainable params: 12,699,519
Non-trainable params: 32
_________________________________________________________________
None
(87050, 5)
train/ABBOTTS BABBLER/001.jpg
(1, 224, 224, 3)
(87050, 5)
train/ABBOTTS BABBLER/001.jpg
(1, 224, 224, 3)


2023-04-14 21:22:04.316024: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-04-14 21:22:06.323041: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-04-14 21:22:07.095241: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-14 21:22:07.403223: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f5faf229690 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-14 21:22:07.403273: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti with Max-Q Design, Compute Capability 7.5
2023-04-14 21:22:07.564565: I tensorflow/tsl/p

1280/1280 [==============================] - ETA: 0s - loss: 6.2863 - accuracy: 0.0024

2023-04-14 21:32:33.721629: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1280/1280 [==============================] - 650s 505ms/step - loss: 6.2863 - accuracy: 0.0024 - val_loss: 6.1958 - val_accuracy: 0.0044
(87050, 5)
train/ABBOTTS BABBLER/001.jpg
(1, 224, 224, 3)


2023-04-14 21:32:55.227605: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


39/39 [==============================] - 23s 580ms/step - loss: 6.1899 - accuracy: 0.0040


In [6]:
cnn_model.save("batch_normalized_3_dense_5_epochs")

INFO:tensorflow:Assets written to: batch_normalized_3_dense_5_epochs/assets


INFO:tensorflow:Assets written to: batch_normalized_3_dense_5_epochs/assets


In [14]:
test_image = Image.open("images to predict/1.jpg").resize((224,224))
image_array = np.expand_dims(np.asarray(test_image),axis=0)
print(image_array)
cnn_model.predict(image_array)

[[[[ 87  79  42]
   [ 90  82  45]
   [ 90  82  46]
   ...
   [ 87  76  54]
   [ 94  84  57]
   [ 95  86  57]]

  [[ 88  80  43]
   [ 90  82  45]
   [ 90  82  46]
   ...
   [ 89  78  56]
   [ 96  86  59]
   [ 94  85  56]]

  [[ 88  80  43]
   [ 89  81  44]
   [ 90  82  46]
   ...
   [ 96  86  61]
   [ 90  78  52]
   [101  89  63]]

  ...

  [[103  92  74]
   [105  94  76]
   [106  94  78]
   ...
   [ 99  86  54]
   [ 97  83  54]
   [ 97  84  52]]

  [[104  92  76]
   [106  94  78]
   [107  95  79]
   ...
   [ 99  86  52]
   [ 96  83  51]
   [ 95  82  48]]

  [[106  94  78]
   [108  96  80]
   [109  97  81]
   ...
   [100  87  53]
   [ 97  84  50]
   [ 96  83  49]]]]
1/1 [==============================] - 0s 28ms/step


array([[2.57067131e-06, 3.99548007e-05, 3.84753780e-03, 1.49096865e-02,
        1.58080012e-02, 5.90787444e-04, 1.26173254e-04, 1.12543837e-03,
        5.93369012e-04, 2.58353393e-04, 8.75800833e-06, 1.16440999e-04,
        3.47768539e-04, 9.35269236e-06, 5.31022029e-04, 3.08653180e-05,
        1.29992513e-05, 2.53940798e-05, 4.21276374e-04, 5.12978341e-03,
        4.32076013e-06, 2.98589410e-04, 3.36751873e-05, 1.32276805e-03,
        1.09392253e-03, 5.11953346e-02, 3.47930938e-04, 2.87048920e-06,
        6.33914350e-03, 4.41652592e-05, 8.55482882e-04, 4.23731096e-03,
        1.99401602e-05, 8.74998950e-05, 5.67223004e-04, 6.12738775e-04,
        3.15797915e-05, 7.62827112e-05, 1.08834956e-06, 3.13567871e-04,
        2.22418574e-03, 6.42655592e-04, 1.77691909e-04, 3.49773956e-03,
        2.67815194e-05, 7.40081945e-04, 1.81456140e-04, 1.90441089e-04,
        5.54084618e-05, 9.02732645e-05, 7.87902437e-03, 1.55617110e-02,
        1.59787363e-04, 2.21194816e-03, 6.60426595e-05, 6.289677

In [3]:
one_epoch_model = tf.keras.models.load_model("base_batch_normalized")
one_epoch_model.summary()

2023-04-03 10:33:53.430591: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-03 10:33:54.065585: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-03 10:33:54.066989: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-03 10:33:54.084907: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-03 10:33:54.085734: I tensorflow/compile

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 222, 222, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 394272)            0         
                                                                 
 dense (Dense)               (None, 511)               201473503 
                                                                 
Total params: 201,474,527
Trainable params: 201,474,463


In [9]:
test_image = Image.open("images to predict/1.jpg").resize((224,224))
image_array = np.expand_dims(np.asarray(test_image),axis=0)
print(image_array)
predict_x = one_epoch_model.predict(image_array)
classes_x = np.argmax(predict_x, axis=1)
print(classes_x)

[[[[ 87  79  42]
   [ 90  82  45]
   [ 90  82  46]
   ...
   [ 87  76  54]
   [ 94  84  57]
   [ 95  86  57]]

  [[ 88  80  43]
   [ 90  82  45]
   [ 90  82  46]
   ...
   [ 89  78  56]
   [ 96  86  59]
   [ 94  85  56]]

  [[ 88  80  43]
   [ 89  81  44]
   [ 90  82  46]
   ...
   [ 96  86  61]
   [ 90  78  52]
   [101  89  63]]

  ...

  [[103  92  74]
   [105  94  76]
   [106  94  78]
   ...
   [ 99  86  54]
   [ 97  83  54]
   [ 97  84  52]]

  [[104  92  76]
   [106  94  78]
   [107  95  79]
   ...
   [ 99  86  52]
   [ 96  83  51]
   [ 95  82  48]]

  [[106  94  78]
   [108  96  80]
   [109  97  81]
   ...
   [100  87  53]
   [ 97  84  50]
   [ 96  83  49]]]]
1/1 [==============================] - 0s 47ms/step
[507]


In [17]:
# Generate class labels
csv_df = pd.read_csv("birds.csv")
dataset = pd.DataFrame(csv_df).to_numpy()
labels = dataset[:,2]
class_ids = dataset[:,0]

print(labels.shape)
print(class_ids.shape)

label_mapping = {}
id_to_name = dict(zip(class_ids, labels))
print(id_to_name)
print(id_to_name[1])

(87050,)
(87050,)
{0.0: 'ABBOTTS BABBLER', 1.0: 'ABBOTTS BOOBY', 2.0: 'ABYSSINIAN GROUND HORNBILL', 3.0: 'AFRICAN CROWNED CRANE', 4.0: 'AFRICAN EMERALD CUCKOO', 5.0: 'AFRICAN FIREFINCH', 6.0: 'AFRICAN OYSTER CATCHER', 7.0: 'AFRICAN PIED HORNBILL', 8.0: 'AFRICAN PYGMY GOOSE', 9.0: 'ALBATROSS', 10.0: 'ALBERTS TOWHEE', 11.0: 'ALEXANDRINE PARAKEET', 12.0: 'ALPINE CHOUGH', 13.0: 'ALTAMIRA YELLOWTHROAT', 14.0: 'AMERICAN AVOCET', 15.0: 'AMERICAN BITTERN', 16.0: 'AMERICAN COOT', 17.0: 'AMERICAN FLAMINGO', 18.0: 'AMERICAN GOLDFINCH', 19.0: 'AMERICAN KESTREL', 20.0: 'AMERICAN PIPIT', 21.0: 'AMERICAN REDSTART', 22.0: 'AMERICAN ROBIN', 23.0: 'AMERICAN WIGEON', 24.0: 'AMETHYST WOODSTAR', 25.0: 'ANDEAN GOOSE', 26.0: 'ANDEAN LAPWING', 27.0: 'ANDEAN SISKIN', 28.0: 'ANHINGA', 29.0: 'ANIANIAU', 30.0: 'ANNAS HUMMINGBIRD', 31.0: 'ANTBIRD', 32.0: 'ANTILLEAN EUPHONIA', 33.0: 'APAPANE', 34.0: 'APOSTLEBIRD', 35.0: 'ARARIPE MANAKIN', 36.0: 'ASHY STORM PETREL', 37.0: 'ASHY THRUSHBIRD', 38.0: 'ASIAN CRESTED IBIS

In [19]:
print(id_to_name[classes_x[0]])

WOODLAND KINGFISHER
